# Scraper La Voz del Sur 

In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from urllib.parse import urljoin

# Configuración mejorada
BASE_URL = "https://www.lavozdelsur.com.mx/"
OUTPUT_FILE = "articulos_lavozdelsur.csv"
DELAY_RANGE = (2, 5)  # Retardo entre 2 y 5 segundos
MAX_RETRIES = 3  # Intentos máximos por artículo

# Headers mejorados para evitar bloqueos
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9',
    'Referer': 'https://www.google.com/',
    'DNT': '1'
}

def get_with_retry(url, retries=MAX_RETRIES):
    """Función con reintentos para mayor robustez"""
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=HEADERS)
            response.raise_for_status()
            return response
        except Exception as e:
            if attempt == retries - 1:
                raise
            wait = random.uniform(5, 10)
            print(f"Intento {attempt + 1} fallado. Esperando {wait:.1f} segundos...")
            time.sleep(wait)

def clean_text(text):
    """Limpia el texto eliminando espacios innecesarios"""
    return ' '.join(text.strip().split()) if text else ""

def extract_article_data(url):
    """Extrae los datos principales de un artículo"""
    try:
        response = get_with_retry(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraer título
        title = clean_text(soup.find('h1', class_='omc-post-heading-standard').get_text())
        
        # Extraer fecha
        date_tag = soup.find('p', class_='omc-date-time-inner')
        date = clean_text(date_tag.get_text().split('|')[0].replace('Published on', '')) if date_tag else ""
        
        # Extraer contenido principal
        article = soup.find('article', id='omc-full-article')
        paragraphs = []
        
        if article:
            # Eliminar elementos no deseados
            for element in article(['script', 'style', 'iframe', 'nav', 'footer', 'aside', 
                                 'div.bottomcontainerBox', 'div.yarpp', 'div.omc-related-posts',
                                 'p.omc-single-tags', 'div.omc-authorbox']):
                element.decompose()
            
            # Extraer texto de párrafos y encabezados
            for p in article.find_all(['p', 'h2', 'h3']):
                text = clean_text(p.get_text())
                if text and len(text.split()) > 2:  # Filtrar textos muy cortos
                    paragraphs.append(text)
        
        return {
            'titulo': title,
            'articulo': '\n\n'.join(paragraphs) if paragraphs else "Contenido no disponible",
            'fecha': date,
            'url': url
        }
        
    except Exception as e:
        print(f"Error al procesar {url}: {str(e)}")
        return None

def get_article_links(page_url):
    """Obtiene todos los enlaces a artículos de una página"""
    try:
        response = get_with_retry(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return [h2.a['href'] for article in soup.find_all('article', class_='omc-blog-two') 
                if (h2 := article.find('h2')) and h2.a]
    except Exception as e:
        print(f"Error obteniendo enlaces de {page_url}: {str(e)}")
        return []

def main():
    """Función principal que ejecuta el scraping"""
    with open(OUTPUT_FILE, mode='w', encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['titulo', 'articulo', 'fecha', 'url'])
        writer.writeheader()
        
        for page_num in range(1, 133):  # Páginas 1 a 132
            page_url = BASE_URL if page_num == 1 else f"{BASE_URL}page/{page_num}/"
            print(f"\n{'='*50}\nProcesando página {page_num}: {page_url}\n{'='*50}")
            
            article_links = get_article_links(page_url)
            
            if not article_links:
                print("No se encontraron artículos en esta página")
                continue
            
            for i, link in enumerate(article_links, 1):
                print(f"\nArtículo {i}/{len(article_links)}:")
                print(f"URL: {link}")
                
                if article_data := extract_article_data(link):
                    writer.writerow(article_data)
                    csvfile.flush()
                    print(f"✓ Título: {article_data['titulo']}")
                    print(f"✓ Fecha: {article_data['fecha']}")
                    print(f"✓ Caracteres extraídos: {len(article_data['articulo'])}")
                
                # Espera aleatoria entre artículos
                wait = random.uniform(*DELAY_RANGE)
                print(f"Esperando {wait:.1f} segundos...")
                time.sleep(wait)
            
            # Espera adicional entre páginas
            page_wait = random.uniform(DELAY_RANGE[0] * 1.5, DELAY_RANGE[1] * 1.5)
            print(f"\nEsperando {page_wait:.1f} segundos antes de la siguiente página...")
            time.sleep(page_wait)

if __name__ == "__main__":
    print("""
    ====================================
    SCRAPER LA VOZ DEL SUR - INICIANDO
    ====================================
    """)
    start_time = time.time()
    
    try:
        main()
    except KeyboardInterrupt:
        print("\nScraping detenido manualmente")
    except Exception as e:
        print(f"\nError inesperado: {str(e)}")
    finally:
        duration = time.time() - start_time
        print(f"""
    ====================================
    SCRAPER COMPLETADO
    Tiempo total: {duration//60:.0f}m {duration%60:.0f}s
    Datos guardados en: {OUTPUT_FILE}
    ====================================
        """)


    SCRAPER LA VOZ DEL SUR - INICIANDO
    

Procesando página 1: https://www.lavozdelsur.com.mx/

Artículo 1/8:
URL: https://www.lavozdelsur.com.mx/dan-banderazo-de-arranque-al-operativo-de-semana-santa-y-pascua-2025-en-jalisco/
✓ Título: Dan Banderazo de Arranque al Operativo de Semana Santa y Pascua 2025 en Jalisco
✓ Fecha: abril 12th, 2025
✓ Caracteres extraídos: 5707
Esperando 2.1 segundos...

Artículo 2/8:
URL: https://www.lavozdelsur.com.mx/la-falta-de-vigilancia-en-las-carreteras/
✓ Título: La Falta de Vigilancia en las Carreteras
✓ Fecha: abril 11th, 2025
✓ Caracteres extraídos: 2823
Esperando 3.3 segundos...

Artículo 3/8:
URL: https://www.lavozdelsur.com.mx/banda-real-de-la-montana-lleva-su-musica-al-festejo-en-honor-a-don-benito-juarez/
✓ Título: Banda Real de la Montaña Lleva su Música al Festejo en Honor a Don Benito Juárez
✓ Fecha: abril 11th, 2025
✓ Caracteres extraídos: 2684
Esperando 2.1 segundos...

Artículo 4/8:
URL: https://www.lavozdelsur.com.mx/muere-un-hombre-a